In [1]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re
import sqlite3

# Function to install packages
def install_packages():
    # Use --upgrade and --no-cache-dir for more robust installation
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--no-cache-dir", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()
# Import after installation
from pyngrok import ngrok
# --- Create Streamlit Config for Dark Theme ---
os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#4F8BF9"
backgroundColor="#0E1117"
secondaryBackgroundColor="#262730"
textColor="#FAFAFA"
font="sans serif"
[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)
print("Applied Dark Theme configuration.")

Installing required packages...
Applied Dark Theme configuration.


In [2]:
# --- Create the Streamlit App File ---
app_code = """
import streamlit as st
import jwt
import datetime
import time
import re

if 'jwt_token' not in st.session_state:
    st.session_state['jwt_token'] = None

if 'page' not in st.session_state:
    st.session_state['page'] = 'login'

if 'users' not in st.session_state:
    st.session_state['users'] = {}

if 'usernames' not in st.session_state:
    st.session_state['usernames'] = set()



# --- Configuration ---
SECRET_KEY = "super_secret_key_for_demo"  # In production, use environment variable
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
# --- JWT Utils ---
def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.datetime.utcnow() + datetime.timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt
def verify_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        return None
    except jwt.InvalidTokenError:
        return None
# --- Validation Utils ---
def is_valid_email(email):
    # Regex for standard email format
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'
    try:
        if re.match(pattern, email):
            return True
    except:
        return False
    return False
def is_valid_password(password):
    # Alphanumeric check and min length 8
    if len(password) < 8:
        return False
    if not password.isalnum():
        return False
    return True
# --- Session State Management ---
if 'jwt_token' not in st.session_state:
    st.session_state['jwt_token'] = None
if 'page' not in st.session_state:
    st.session_state['page'] = 'login'
# Mock Database (In-memory for demo)
# Structure: {email: {'password': password, 'username': username, ...}}
# Also store usernames separately for quick check: {username: email}


# --- Styling ---
st.set_page_config(page_title="Policy Nav Login", page_icon="🤖", layout="wide")

st.markdown(\"""
    <style>
        .stApp {
              background: radial-gradient(ellipse at 50% -20%, rgba(59,130,246,0.15), transparent 60%),
radial-gradient(circle at 20% 10%, rgba(59,130,246,0.08), transparent 40%), linear-gradient(180deg, #0b1220 0%, #0f172a 100%);
        }


        /* Gradient Animation */
@keyframes gradientBG {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}


div[data-testid="stForm"] {
        background: rgba(17, 24, 39, 0.85);
    backdrop-filter: blur(20px);
    padding: 40px;
    border-radius: 16px;
    border: 1px solid rgba(59,130,246,0.2);
    box-shadow: 0 25px 60px rgba(0, 0, 0, 0.6);
}


@keyframes float {
    0% { transform: translateY(0) rotate(0deg); }
    100% { transform: translateY(-120vh) rotate(360deg); }
}

div[data-testid="stSidebar"] {
    background: linear-gradient(180deg, #111827 0%, #0b1220 100%);
    border-right: 1px solid #1f2937;
}


        h1 {
             text-align: center;
    font-size: 2.2rem;
    font-weight: 700;
    background: linear-gradient(90deg, #ffffff, #93c5fd);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
        }
        h3 {
            text-align: center;
            color: #94a3b8;
            font-weight: 300;
            margin-top: 0;
            font-size: 1.2rem;
        }
        /* Main Streamlit Button */

        .stButton > button {
    width: 100%;
    height: 3em;
    border-radius: 30px;
    background: rgba(50, 15, 15, 0.05);
    box-shadow: 0 15px 15px rgba(0, 0, 0, 0.3);
    border-bottom: 1px solid rgba(255, 255, 255, 0.1);
    border-top: 1px solid rgba(255, 255, 255, 0.1);
    color: white;
    font-weight: 400;
    letter-spacing: 1px;
    backdrop-filter: blur(15px);
    transition: 0.5s;
    position: relative;
    overflow: hidden;
}

/* Hover Effect */
.stButton > button:hover {
    letter-spacing: 3px;
}

/* Shine Effect */
.stButton > button::before {
    content: "";
    position: absolute;
    top: 0;
    left: 0;
    width: 50%;
    height: 100%;
    background: linear-gradient(to left, rgba(255,255,255,0.15), transparent);
    transform: skewX(45deg) translate(0);
    transition: 0.55s;
}

.stButton > button:hover::before {
    transform: skewX(45deg) translate(250px);
}






.chat-box {
    background: rgba(30, 41, 59, 0.85);
    backdrop-filter: blur(15px);
    border-radius: 24px;
    padding: 30px;
    height: 420px;
    overflow-y: auto;
    border: 1px solid rgba(59,130,246,0.15);
    box-shadow: 0 20px 60px rgba(0, 0, 0, 0.6);
}

/* User Message */
.user-msg {
    text-align: right;
     background-color: #0f172a;
    border: 1px solid #1e293b;
    color: white;
    padding: 12px 16px;
    border-radius: 20px 20px 5px 20px;
    margin: 10px;
    display: inline-block;
    max-width: 60%;
    float: right;
    clear: both;
}

/* Bot Message */
.bot-msg {
    text-align: left;
    background-color: #2563eb;
    color: white;
    padding: 12px 16px;
    border-radius: 20px 20px 20px 5px;
    margin: 10px;
    display: inline-block;
    max-width: 60%;
    float: left;
    clear: both;
}



    </style>


\""", unsafe_allow_html=True)


# --- Views ---
def login_page():
    st.markdown("<br>", unsafe_allow_html=True)
    col1, col2, col3 = st.columns([1, 2, 1])

    with col2:
        st.title("Policy Nav LogIn")
        st.markdown("<h3>Please sign in to continue</h3>", unsafe_allow_html=True)

        with st.form("login_form"):
            email = st.text_input("Email Address")
            password = st.text_input("Password", type="password")
            submitted = st.form_submit_button("Sign In")

            if submitted:
                if email in st.session_state['users'] and st.session_state['users'][email]['password'] == password:
                    username = st.session_state['users'][email]['username']
                    token = create_access_token({"sub": email, "username": username})
                    st.session_state['jwt_token'] = token
                    st.success("Login successful!")
                    time.sleep(0.5)
                    st.rerun()
                else:
                    st.error("Invalid email or password")

        st.markdown("---")
        c1, c2 = st.columns(2)
        with c1:
            if st.button("Forgot Password?"):
                st.session_state['page'] = 'forgot'
                st.rerun()
        with c2:
            if st.button("Create an Account"):
                st.session_state['page'] = 'signup'
                st.rerun()
def signup_page():
    st.markdown("<br>", unsafe_allow_html=True)
    col1, col2, col3 = st.columns([1, 2, 1])

    with col2:
        st.title("Create Account")

        with st.form("signup_form"):
            username = st.text_input("Username (Required)")
            email = st.text_input("Email Address (@domain.com required)")
            password = st.text_input("Password (min 8 chars, alphanumeric)")
            confirm_password = st.text_input("Confirm Password", type="password")
            security_question = st.selectbox(
                "Security Question",
                [
                    "What is your pet name?",
                    "What is your mother’s maiden name?",
                    "What is your favorite teacher?"
                ]
            )
            security_answer = st.text_input("Security Answer")
            submitted = st.form_submit_button("Sign Up")

            if submitted:
                errors = []

                # Username Validation
                if not username:
                    errors.append("Username is mandatory.")
                elif username in st.session_state['usernames']:
                    errors.append(f"Username '{username}' is already taken.")

                # Email Validation
                if not email:
                    errors.append("Email is mandatory.")
                elif not is_valid_email(email):
                    errors.append("Invalid Email format (e.g. user@domain.com).")
                elif email in st.session_state['users']:
                    errors.append(f"Email '{email}' is already registered.")

                # Password Validation
                if not password:
                    errors.append("Password is mandatory.")
                elif not is_valid_password(password):
                    errors.append("Password must be at least 8 characters long and contain only alphanumeric characters.")

                # Confirm Password
                if password != confirm_password:
                    errors.append("Passwords do not match.")

                if not security_answer:
                    errors.append("Security Answer is mandatory.")

                if errors:
                    for error in errors:
                        st.error(error)
                else:
                    # Success
                    st.session_state['users'][email] = {'password': password, 'username': username, 'security_question': security_question,'security_answer': security_answer.lower()}
                    st.session_state['usernames'].add(username)

                    # Auto-login after signup
                    token = create_access_token({"sub": email, "username": username})
                    st.session_state['jwt_token'] = token
                    st.success("Account created successfully!")
                    time.sleep(1)
                    st.rerun()

        st.markdown("---")
        if st.button("Back to Login"):
            st.session_state['page'] = 'login'
            st.rerun()

def forgot_password_page():
    st.title("Forgot Password")

    email = st.text_input("Enter your registered Email")

    if st.button("Verify Email"):
        if email in st.session_state['users']:
            st.session_state['reset_email'] = email
            st.success("Email verified!")
        else:
            st.error("Email not found.")

    if 'reset_email' in st.session_state:
        user_data = st.session_state['users'][st.session_state['reset_email']]

        st.write("Security Question:")
        st.info(st.session_state['security_question'])


        answer = st.text_input("Enter Security Answer")

        if st.button("Verify Answer"):
            if answer.lower() == user_data['security_answer']:
                st.session_state['answer_verified'] = True
                st.success("Answer verified!")
            else:
                st.error("Incorrect answer.")

    if st.session_state.get('answer_verified'):
        new_password = st.text_input("Enter New Password", type="password")
        confirm_new_password = st.text_input("Confirm New Password", type="password")

        if st.button("Reset Password"):
            if not is_valid_password(new_password):
                st.error("Password must be alphanumeric and at least 8 characters.")
            elif new_password != confirm_new_password:
                st.error("Passwords do not match.")
            else:
                email = st.session_state['reset_email']
                st.session_state['users'][email]['password'] = new_password
                st.success("Password updated successfully!")
                time.sleep(1)
                st.session_state['page'] = 'login'
                st.session_state.pop('reset_email', None)
                st.session_state.pop('answer_verified', None)
                st.rerun()

    if st.button("Back to Login"):
        st.session_state['page'] = 'login'
        st.rerun()


def dashboard_page():
    token = st.session_state.get('jwt_token')
    payload = verify_token(token)

    if not payload:
        st.session_state['jwt_token'] = None
        st.warning("Session expired or invalid. Please login again.")
        time.sleep(1)
        st.rerun()
        return

    username = payload.get("username", "User")

    # ---------- Initialize Chat ----------
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = [
            {"role": "bot", "content": "Hello! I am LLM. Ask me anything about LLM!"}
        ]

    # ---------- SIDEBAR ----------
    with st.sidebar:
        st.title("🤖 LLM")
        st.markdown("---")

        if st.button("➕ New Chat", use_container_width=True):
            st.session_state.chat_history = [
                {"role": "bot", "content": "Hello! I am LLM. Ask me anything about LLM!"}
            ]
            st.rerun()

        st.markdown("### History")
        st.markdown("- Project analysis")
        st.markdown("- NLP")
        st.markdown("---")

        st.markdown("### Settings")
        if st.button("Logout", use_container_width=True):
            st.session_state['jwt_token'] = None
            st.rerun()

    # ---------- HEADER ----------
    st.markdown(
        f'''
        <div style="text-align:center; margin-top:20px;">
            <h1 style="color:white;">Welcome, {username.upper()}!</h1>
            <h3 style="color:#cbd5e1;">How can I help you today?</h3>
        </div>
        ''',
        unsafe_allow_html=True
    )

    st.markdown("<br>", unsafe_allow_html=True)

    # ---------- CHAT BOX ----------
    chat_html = '<div class="chat-box">'

    for message in st.session_state.chat_history:
        if message["role"] == "user":
            chat_html += f'<div class="user-msg">{message["content"]}</div>'
        else:
            chat_html += f'<div class="bot-msg">{message["content"]}</div>'

    chat_html += '</div>'

    st.markdown(chat_html, unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    # ---------- INPUT SECTION ----------
    with st.form("chat_form", clear_on_submit=True):
        col1, col2 = st.columns([6, 1])

        with col1:
            user_input = st.text_input(
                "",
                placeholder="Ask me anything about LLM...",
                label_visibility="collapsed"
            )

        with col2:
            submit = st.form_submit_button("Send")

        if submit and user_input:
            st.session_state.chat_history.append(
                {"role": "user", "content": user_input}
            )

            st.session_state.chat_history.append(
                {"role": "bot", "content": "I am a demo bot. I received your message!"}
            )

            st.rerun()



# --- Main App Logic ---
token = st.session_state.get('jwt_token')
if token:
    if verify_token(token):
        dashboard_page()
    else:
        st.session_state['jwt_token'] = None
        st.session_state['page'] = 'login'
        st.rerun()
else:
    if st.session_state['page'] == 'signup':
        signup_page()
    elif st.session_state['page'] == 'forgot':
        forgot_password_page()
    else:
        login_page()


"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)
print("Streamlit app code written to 'app.py'")

Streamlit app code written to 'app.py'


In [ ]:
# --- Wait for Streamlit to Start ---
def wait_for_streamlit(port=8501, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                sock.close()
                return True
            sock.close()
        except Exception:
            pass
        time.sleep(1)
    return False

import os
import subprocess
import time
import socket
from pyngrok import ngrok

# --- Ngrok Setup ---
print("\nTo access the app, you need an Ngrok Authtoken.")
print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
authtoken = input("Enter your Ngrok Authtoken: ").strip()
if authtoken:
    ngrok.set_auth_token(authtoken)

    # Kill any existing ngrok process
    os.system("pkill ngrok")
    os.system("pkill streamlit")

    # Run Streamlit in the background FIRST
    print("Starting Streamlit...")
    # Using Subprocess.Popen to run in background
    # Redirecting output to /dev/null to keep cell clean
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "localhost"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Wait for it to be ready
    if wait_for_streamlit():
        print("Streamlit is active! Connecting Ngrok...")
        # Open a tunnel to the streamlit port 8501
        try:
            public_url = ngrok.connect(8501).public_url
            print(f"\n🚀 Streamlit App is running!")
            print(f"👉 Public URL: {public_url}")
            print("\n(Click the URL above to open the app)")

            # Keep main thread alive
            try:
                # Keep checking if process is alive
                while process.poll() is None:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("Stopping...")
                ngrok.disconnect(public_url)
                process.terminate()
        except Exception as e:
            print(f"Ngrok connection failed: {e}")
            process.terminate()
    else:
        print("Error: Streamlit failed to start in time.")
        process.terminate()
else:
    print("Ngrok Authtoken is required to expose the app publicly.")


To access the app, you need an Ngrok Authtoken.
Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter your Ngrok Authtoken: 39X6gUVNcQM2EpdlTp5yjdqqscC_6TYJYkqXU7S1ayoZ1ZFV
Starting Streamlit...
Streamlit is active! Connecting Ngrok...

🚀 Streamlit App is running!
👉 Public URL: https://unparadoxal-constrictedly-lurline.ngrok-free.dev

(Click the URL above to open the app)
